# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$
5. Train a DNN to predict the returns of bitcoin with GD.  Since this has random initialization then each prediction of a given $x$ is stochastic...We learn the distribution of this conditional RV (conditioned on x in the input space).
$$
Y_x \triangleq \hat{f}_{\theta_{T}}(x), \qquad \theta_{(t+1)}\triangleq \theta_{(t)} + \lambda \sum_{x \in \mathbb{X}} \nabla_{\theta}\|\hat{f}_{\theta_t}(x) - f(x)\|, \qquad \theta_0 \sim N_d(0,1);
$$
$T\in \mathbb{N}$ is a fixed number of "SGD" iterations (typically identified by cross-validation on a single SGD trajectory for a single initialization) and where $\theta \in \mathbb{R}^{(d_{J}+1)+\sum_{j=0}^{J-1} (d_{j+1}d_j + 1)}$ and $d_j$ is the dimension of the "bias" vector $b_j$ defining each layer of the DNN with layer dimensions:
$$
\hat{f}_{\theta}(x)\triangleq A^{(J)}x^{(J)} + b^{(J)},\qquad x^{(j+1)}\triangleq \sigma\bullet A^{j}x^{(j)} + b^{j},\qquad x^{(0)}\triangleq x
.
$$

#### Mode:
Software/Hardware Testing or Real-Deal?

In [38]:
trial_run = True

### Simulation Method:

In [39]:
# Random DNN
# f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# Random DNN internal noise
f_unknown_mode = "DNN_with_Random_Weights"
Depth_Bayesian_DNN = 1
width = 5

# Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.1

# GD with Randomized Input
# f_unknown_mode = "GD_with_randomized_input"
GD_epochs = 50

# SDE with fractional Driver
#f_unknown_mode = "Rough_SDE"
N_Euler_Steps = 10**2
Hurst_Exponent = 0.75

# f_unknown_mode = "Rough_SDE_Vanilla"
## Define Process' dynamics in (2) cell(s) below.

## Problem Dimension

In [40]:
problem_dim = 1

#### Vanilla fractional SDE:
If f_unknown_mode == "Rough_SDE_Vanilla" is selected, then we can specify the process's dynamics.  

In [41]:
#--------------------------#
# Define Process' Dynamics #
#--------------------------#
drift_constant = 0.1
volatility_constant = 0.01

# Define DNN Applier
def f_unknown_drift_vanilla(x):
    x_internal = x
    x_internal = drift_constant*x_internal
    return x_internal
def f_unknown_vol_vanilla(x):
    x_internal = volatility_constant*np.diag(np.ones(problem_dim))
    return x_internal

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [42]:
train_test_ratio = .2
N_train_size = 10

Monte-Carlo Paramters

In [43]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**5

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [44]:
# Hyper-parameters of Cover
delta = 0.1
Proportion_per_cluster = .75

## Dependencies and Auxiliary Script(s)

In [45]:
# %run Loader.ipynb
exec(open('Loader.py').read())
# Load Packages/Modules
exec(open('Init_Dump.py').read())
import time as time #<- Note sure why...but its always seems to need 'its own special loading...'

Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready


# Simulate or Parse Data

In [46]:
# %run Data_Simulator_and_Parser.ipynb
exec(open('Data_Simulator_and_Parser.py').read())

  0%|          | 0/10 [00:00<?, ?it/s]

---------------------------------------
Beginning Data-Parsing/Simulation Phase
---------------------------------------
Deciding on Which Simulator/Parser To Load
Setting/Defining: Internal Parameters
Deciding on Which Type of Data to Get/Simulate
Simulating Output Data for given input data


100%|██████████| 2/2 [00:08<00:00,  4.25s/it]

----------------------------------
Done Data-Parsing/Simulation Phase
----------------------------------


#### Scale Data
This is especially important to avoid exploding gradient problems when training the ML-models.

In [62]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Run Main:

In [63]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
# %run Universal_Measure_Valued_Networks_Backend.ipynb
exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

100%|██████████| 8/8 [00:00<00:00, 16170.81it/s]

------------------------------
Running script for main model!
------------------------------
Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.8s finished


Epoch 1/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0729 - accuracy: 0.1000
Epoch 2/200
1/1 [==============================] - 0s 1ms/step - loss: 2.0633 - accuracy: 0.1000
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0544 - accuracy: 0.1000
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 2.0456 - accuracy: 0.1000
Epoch 5/200
1/1 [==============================] - 0s 832us/step - loss: 2.0373 - accuracy: 0.1000
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0295 - accuracy: 0.1000
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0219 - accuracy: 0.1000
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 2.0139 - accuracy: 0.1000
Epoch 9/200
1/1 [==============================] - 0s 1ms/step - loss: 2.0063 - accuracy: 0.1000
Epoch 10/200
1/1 [==============================] - 0s 2ms/step - loss: 1.9988 - accuracy: 0.1000
Epoch 11/200
1/1 [=========

1/1 [==============================] - 0s 1ms/step - loss: 1.3018 - accuracy: 0.5000
Epoch 85/200
1/1 [==============================] - 0s 2ms/step - loss: 1.2907 - accuracy: 0.5000
Epoch 86/200
1/1 [==============================] - 0s 1ms/step - loss: 1.2796 - accuracy: 0.5000
Epoch 87/200
1/1 [==============================] - 0s 2ms/step - loss: 1.2686 - accuracy: 0.5000
Epoch 88/200
1/1 [==============================] - 0s 984us/step - loss: 1.2578 - accuracy: 0.5000
Epoch 89/200
1/1 [==============================] - 0s 1ms/step - loss: 1.2469 - accuracy: 0.5000
Epoch 90/200
1/1 [==============================] - 0s 2ms/step - loss: 1.2362 - accuracy: 0.5000
Epoch 91/200
1/1 [==============================] - 0s 3ms/step - loss: 1.2255 - accuracy: 0.5000
Epoch 92/200
1/1 [==============================] - 0s 870us/step - loss: 1.2148 - accuracy: 0.6000
Epoch 93/200
1/1 [==============================] - 0s 1ms/step - loss: 1.2043 - accuracy: 0.6000
Epoch 94/200
1/1 [===========

...

#--------------------#
 Get Training Error(s)
#--------------------#


  0%|          | 0/2 [00:00<?, ?it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#


100%|██████████| 2/2 [00:00<00:00,  3.28it/s]

#------------------------#
 Get Testing Error(s): END
#------------------------#
                                        DNM  MC-Oracle
W1-95L                         3.326321e-07   0.000000
W1                             6.896845e-07   0.000000
W1-95R                         1.046737e-06   0.000000
M-95L                          1.290718e-05   0.000013
M                              4.281266e-05   0.000043
M-95R                          7.271814e-05   0.000073
N_Par                          4.220800e+04   0.000000
Train_Time                     1.104827e+01  53.664175
Test_Time/MC-Oracle_Test_Time  5.298994e-03   1.000000
------------------------------------
Done: Running script for main model!
------------------------------------


---
# Run: All Benchmarks

## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [65]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


100%|██████████| 2/2 [00:00<00:00, 63.15it/s]

---------------------
Training: ENET - Done
---------------------
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0636s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s


                                        DNM  MC-Oracle         ENET KRidge  \
W1-95L                             0.000002   0.000000            -      -   
W1                                 0.000005   0.000000            -      -   
W1-95R                             0.000009   0.000000            -      -   
M-95L                              0.000092   0.000080  9.15661e-05    NaN   
M                                  0.000145   0.000145  0.000168421    NaN   
M-95R                              0.000199   0.000185  0.000236557    NaN   
N_Par                          42208.000000   0.000000           20    NaN   
Train_Time                        11.048274  53.664175  1.62012e+09    NaN   
Test_Time/MC-Oracle_Test_Time      0.005299   1.000000  6.68157e-06    NaN   

                              GBRF  DNN  
W1-95L                           -    -  
W1                               -    -  
W1-95R                           -    -  
M-95L                          NaN  NaN  
M        

...

#------------#
 Get Error(s) 
#------------#


  0%|          | 0/2 [00:00<?, ?it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#


100%|██████████| 2/2 [00:00<00:00, 54.91it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
Updated DataFrame
                                        DNM  MC-Oracle         ENET  \
W1-95L                             0.000002   0.000000            -   
W1                                 0.000005   0.000000            -   
W1-95R                             0.000009   0.000000            -   
M-95L                              0.000092   0.000080  9.15661e-05   
M                                  0.000145   0.000145  0.000168421   
M-95R                              0.000199   0.000185  0.000236557   
N_Par                          42208.000000   0.000000           20   
Train_Time                        11.048274  53.664175  1.62012e+09   
Test_Time/MC-Oracle_Test_Time      0.005299   1.000000  6.68157e-06   

                                    KRidge         GBRF         DNN  
W1-95L                                   -            -           -  
W1                                       -            -           -  
W

# Summary of Point-Mass Regression Models

#### Training Model Facts

In [66]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                                        DNM  MC-Oracle         ENET  \
W1-95L                             0.000002   0.000000            -   
W1                                 0.000005   0.000000            -   
W1-95R                             0.000009   0.000000            -   
M-95L                              0.000092   0.000080  9.15661e-05   
M                                  0.000145   0.000145  0.000168421   
M-95R                              0.000199   0.000185  0.000236557   
N_Par                          42208.000000   0.000000           20   
Train_Time                        11.048274  53.664175  1.62012e+09   
Test_Time/MC-Oracle_Test_Time      0.005299   1.000000  6.68157e-06   

                                    KRidge         GBRF         DNN  
W1-95L                                   -            -           -  
W1                                       -            -           -  
W1-95R                                   -            -           -  
M-95L    

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,0.000002,0.000000,-,-,-,-
W1,0.000005,0.000000,-,-,-,-
W1-95R,0.000009,0.000000,-,-,-,-
M-95L,0.000092,0.000080,9.15661e-05,0.000107537,7.06454e-05,0.0228225
M,0.000145,0.000145,0.000168421,0.000169909,0.000174036,0.0280817
M-95R,0.000199,0.000185,0.000236557,0.000233372,0.000253984,0.0324094
N_Par,42208.000000,0.000000,20,0,1000,40801
Train_Time,11.048274,53.664175,1.62012e+09,0.561581,0.779469,7.5179
Test_Time/MC-Oracle_Test_Time,0.005299,1.000000,6.68157e-06,2.19537e-05,6.77703e-05,0.00706781


#### Testing Model Facts

In [68]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                                        DNM  MC-Oracle          ENET  \
W1-95L                         3.326321e-07   0.000000  3.269980e-03   
W1                             6.896845e-07   0.000000  3.307251e-03   
W1-95R                         1.046737e-06   0.000000  3.344523e-03   
M-95L                          1.290718e-05   0.000013  1.759886e-05   
M                              4.281266e-05   0.000043  4.789610e-05   
M-95R                          7.271814e-05   0.000073  7.819334e-05   
N_Par                          4.220800e+04   0.000000  2.000000e+01   
Train_Time                     1.104827e+01  53.664175  1.620115e+09   
Test_Time/MC-Oracle_Test_Time  5.298994e-03   1.000000  6.681574e-06   

                                 KRidge         GBRF           DNN  
W1-95L                         0.003270     0.003270      0.003582  
W1                             0.003307     0.003307      0.003621  
W1-95R                         0.003345     0.003345      0.003660  
M-9

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN
W1-95L,3.326321e-07,0.000000,3.269980e-03,0.003270,0.003270,0.003582
W1,6.896845e-07,0.000000,3.307251e-03,0.003307,0.003307,0.003621
W1-95R,1.046737e-06,0.000000,3.344523e-03,0.003345,0.003345,0.003660
M-95L,1.290718e-05,0.000013,1.759886e-05,0.000009,0.000070,0.017658
M,4.281266e-05,0.000043,4.789610e-05,0.000039,0.000100,0.017717
M-95R,7.271814e-05,0.000073,7.819334e-05,0.000068,0.000130,0.017775
N_Par,4.220800e+04,0.000000,2.000000e+01,0.000000,1000.000000,40801.000000
Train_Time,1.104827e+01,53.664175,1.620115e+09,0.561581,0.779469,7.517898
Test_Time/MC-Oracle_Test_Time,5.298994e-03,1.000000,6.681574e-06,0.000022,0.000068,0.007068


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\Sigma}(x)\hat{\Sigma}^{\top})\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to 
(2\pi)^{-\frac{d}{2}}\det(\hat{\Sigma}(x))^{-\frac{1}{2}} \, e^{ -\frac{1}{2}(\cdot - \hat{\mu}(x))^{{{\!\mathsf{T}}}} \hat{\Sigma}(x)^{-1}(\cdot - \hat{\mu}(x)) } \mu \in \mathcal{G}_d\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [69]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.7s finished
  0%|          | 0/10 [00:00<?, ?it/s]

Infering Parameters for Deep Gaussian Network to train on!


100%|██████████| 10/10 [00:04<00:00,  2.15it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Done Getting Parameters for Deep Gaussian Network!
Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits


...

1/1 [==============================] - 0s 2ms/step - loss: 0.4584 - mse: 0.4049 - mae: 0.4584 - mape: 7654.9453
Epoch 195/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4584 - mse: 0.4050 - mae: 0.4584 - mape: 7599.5264
Epoch 196/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4584 - mse: 0.4051 - mae: 0.4584 - mape: 7544.3916
Epoch 197/200
1/1 [==============================] - 0s 1ms/step - loss: 0.4584 - mse: 0.4052 - mae: 0.4584 - mape: 7489.5376
Epoch 198/200
1/1 [==============================] - 0s 1ms/step - loss: 0.4584 - mse: 0.4053 - mae: 0.4584 - mape: 7434.9639
Epoch 199/200
1/1 [==============================] - 0s 3ms/step - loss: 0.4584 - mse: 0.4054 - mae: 0.4584 - mape: 7380.6743
Epoch 200/200
1/1 [==============================] - 0s 4ms/step


  0%|          | 0/10 [00:00<?, ?it/s]

Training Deep Gaussian Network!: END
#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


100%|██████████| 2/2 [00:00<00:00, 14.10it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------------------------#
 Get Testing Errors for: Gaussian Models
#--------------------------------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
                                        DNM  MC-Oracle         ENET  \
W1-95L                             0.000002   0.000000            -   
W1                                 0.000005   0.000000            -   
W1-95R                             0.000009   0.000000            -   
M-95L                              0.000092   0.000080  9.15661e-05   
M                                  0.000145   0.000145  0.000168421   
M-95R                              0.000199   0.000185  0.000236557   
N_Par                          42208.000000   0.000000        

In [70]:
print("Prediction Quality (Updated): Test")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Prediction Quality (Updated): Test
                                        DNM  MC-Oracle          ENET  \
W1-95L                         3.326321e-07   0.000000  3.269980e-03   
W1                             6.896845e-07   0.000000  3.307251e-03   
W1-95R                         1.046737e-06   0.000000  3.344523e-03   
M-95L                          1.290718e-05   0.000013  1.759886e-05   
M                              4.281266e-05   0.000043  4.789610e-05   
M-95R                          7.271814e-05   0.000073  7.819334e-05   
N_Par                          4.220800e+04   0.000000  2.000000e+01   
Train_Time                     1.104827e+01  53.664175  1.620115e+09   
Test_Time/MC-Oracle_Test_Time  5.298994e-03   1.000000  6.681574e-06   

                                 KRidge         GBRF           DNN       GPR  \
W1-95L                         0.003270     0.003270      0.003582  0.002923   
W1                             0.003307     0.003307      0.003621  0.002958   
W1-9

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN
W1-95L,3.326321e-07,0.000000,3.269980e-03,0.003270,0.003270,0.003582,0.002923,0.868309
W1,6.896845e-07,0.000000,3.307251e-03,0.003307,0.003307,0.003621,0.002958,0.873949
W1-95R,1.046737e-06,0.000000,3.344523e-03,0.003345,0.003345,0.003660,0.002993,0.879589
M-95L,1.290718e-05,0.000013,1.759886e-05,0.000009,0.000070,0.017658,0.000007,0.018154
M,4.281266e-05,0.000043,4.789610e-05,0.000039,0.000100,0.017717,0.000030,0.018198
M-95R,7.271814e-05,0.000073,7.819334e-05,0.000068,0.000130,0.017775,0.000052,0.018242
N_Par,4.220800e+04,0.000000,2.000000e+01,0.000000,1000.000000,40801.000000,0.000000,40801.000000
Train_Time,1.104827e+01,53.664175,1.620115e+09,0.561581,0.779469,7.517898,0.730688,5.024743
Test_Time/MC-Oracle_Test_Time,5.298994e-03,1.000000,6.681574e-06,0.000022,0.000068,0.007068,0.000036,0.007340


In [71]:
print("Prediction Quality (Updated): Train")
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

Prediction Quality (Updated): Train
                                        DNM  MC-Oracle         ENET  \
W1-95L                             0.000002   0.000000            -   
W1                                 0.000005   0.000000            -   
W1-95R                             0.000009   0.000000            -   
M-95L                              0.000092   0.000080  9.15661e-05   
M                                  0.000145   0.000145  0.000168421   
M-95R                              0.000199   0.000185  0.000236557   
N_Par                          42208.000000   0.000000           20   
Train_Time                        11.048274  53.664175  1.62012e+09   
Test_Time/MC-Oracle_Test_Time      0.005299   1.000000  6.68157e-06   

                                    KRidge         GBRF         DNN       GPR  \
W1-95L                                   -            -           -  0.004363   
W1                                       -            -           -  0.007077   
W1-95R    

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN
W1-95L,0.000002,0.000000,-,-,-,-,0.004363,0.868309
W1,0.000005,0.000000,-,-,-,-,0.007077,0.873949
W1-95R,0.000009,0.000000,-,-,-,-,0.009855,0.879589
M-95L,0.000092,0.000080,9.15661e-05,0.000107537,7.06454e-05,0.0228225,0.000118,0.018154
M,0.000145,0.000145,0.000168421,0.000169909,0.000174036,0.0280817,0.000198,0.018198
M-95R,0.000199,0.000185,0.000236557,0.000233372,0.000253984,0.0324094,0.000273,0.018242
N_Par,42208.000000,0.000000,20,0,1000,40801,0.000000,40801.000000
Train_Time,11.048274,53.664175,1.62012e+09,0.561581,0.779469,7.5179,0.730688,5.024743
Test_Time/MC-Oracle_Test_Time,0.005299,1.000000,6.68157e-06,2.19537e-05,6.77703e-05,0.00706781,0.000036,0.007340


# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [72]:
if output_dim == 1:
    # %run Mixture_Density_Network.ipynb
    exec(open('Mixture_Density_Network.py').read())

  0%|          | 0/10 [00:00<?, ?it/s]

Preparing Training Outputs for MDNs using EM-Algorithm


100%|██████████| 10/10 [00:11<00:00,  1.17s/it]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Prepared Training Outputs for MDNs using EM-Algorithm!
Deep Feature Builder - Ready
(0)
Training Mixture Density Network (MDN): Means: Start!
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    4.6s finished


...

Training Mixture Density Network (MDN): Mixture Coefficients: END!
#--------------------#
 Get Training Error(s)
#--------------------#


  0%|          | 0/2 [00:00<?, ?it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------#
 Get Test Error(s)
#--------------------#


100%|██████████| 2/2 [00:00<00:00,  3.49it/s]

#---------------------#
 Get Test Error(s): END
#---------------------#
#---------------------------#
 Get Training Error(s): Begin
#---------------------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#
#--------------------------#
 Get Testing Error(s): Begin
#--------------------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
Updated DataFrame
-------------------------------------------------
Train
                                        DNM  MC-Oracle         ENET  \
W1-95L                             0.000002   0.000000            -   
W1                                 0.000005   0.000000            -   
W1-95R                             0.000009   0.000000            -   
M-95L                              0.000092   0.000080  9.15661e-05   
M      

## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

---
# Final Results
---

## Prasing Quality Metric Results

#### Finalizing Saving

In [73]:
## Write Performance Metrics
Summary_pred_Qual_models.to_latex((results_tables_path+"/Final_Results/"+"Performance_metrics_Problem_Type_"+str(f_unknown_mode)+"Problemdimension"+str(problem_dim)+"__SUMMARY_METRICS.tex"),
                                 caption=("Quality Metrics; d:"+str(problem_dim)+", D:"+str(output_dim)+", Depth:"+str(Depth_Bayesian_DNN)+", Width:"+str(width)+", Dropout rate:"+str(Dropout_rate)+"."),
                                 float_format="{:0.3g}".format)

# For Terminal Runner(s):

In [74]:
# For Terminal Running
print("===================")
print("Predictive Quality:")
print("===================")
print(Summary_pred_Qual_models)
print("===================")
print(" ")
print(" ")
print(" ")
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))
print("🙃🙃 Have a wonderful day! 🙃🙃")
Summary_pred_Qual_models

Predictive Quality:
                                        DNM  MC-Oracle         ENET  \
W1-95L                             0.000002   0.000000            -   
W1                                 0.000005   0.000000            -   
W1-95R                             0.000009   0.000000            -   
M-95L                              0.000092   0.000080  9.15661e-05   
M                                  0.000145   0.000145  0.000168421   
M-95R                              0.000199   0.000185  0.000236557   
N_Par                          42208.000000   0.000000           20   
Train_Time                        11.048274  53.664175  1.62012e+09   
Test_Time/MC-Oracle_Test_Time      0.005299   1.000000  6.68157e-06   

                                    KRidge         GBRF         DNN       GPR  \
W1-95L                                   -            -           -  0.004363   
W1                                       -            -           -  0.007077   
W1-95R                    

,DNM,MC-Oracle,ENET,KRidge,GBRF,DNN,GPR,DGN,MDN
W1-95L,0.000002,0.000000,-,-,-,-,0.004363,0.868309,0.825095
W1,0.000005,0.000000,-,-,-,-,0.007077,0.873949,0.826658
W1-95R,0.000009,0.000000,-,-,-,-,0.009855,0.879589,0.828222
M-95L,0.000092,0.000080,9.15661e-05,0.000107537,7.06454e-05,0.0228225,0.000118,0.018154,0.004934
M,0.000145,0.000145,0.000168421,0.000169909,0.000174036,0.0280817,0.000198,0.018198,0.007488
M-95R,0.000199,0.000185,0.000236557,0.000233372,0.000253984,0.0324094,0.000273,0.018242,0.009666
N_Par,42208.000000,0.000000,20,0,1000,40801,0.000000,40801.000000,126624.000000
Train_Time,11.048274,53.664175,1.62012e+09,0.561581,0.779469,7.5179,0.730688,5.024743,0.146728
Test_Time/MC-Oracle_Test_Time,0.005299,1.000000,6.68157e-06,2.19537e-05,6.77703e-05,0.00706781,0.000036,0.007340,3.969606


---
# Fin
---

---